In [1]:
from ogb.nodeproppred import PygNodePropPredDataset
import os
import os.path as osp
import sys
import torch_geometric
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import InMemoryDataset, download_url

sys.path.append(osp.abspath(".."))
print(sys.path)

from utils.encoder import SentenceEncoder
from utils_data.custom_pyg import CustomPygDataset
from utils.dataloader import GetDataloader

['/home/prateek/graphs-with-llms-experiments/utils_data', '/home/prateek/miniconda3/envs/torch_pyg/lib/python310.zip', '/home/prateek/miniconda3/envs/torch_pyg/lib/python3.10', '/home/prateek/miniconda3/envs/torch_pyg/lib/python3.10/lib-dynload', '', '/home/prateek/miniconda3/envs/torch_pyg/lib/python3.10/site-packages', '/home/prateek/graphs-with-llms-experiments']


In [2]:
# Loading the pubmed dataset
data_root = "../data"

In [3]:
class PubmedPyGDataset(InMemoryDataset):
    def __init__(self, dataRoot="../data", custom_dataRoot="../custom_data", sentence_encoder=None, transform=None, pre_transform=None, pre_filter=None):
        self.data_root = dataRoot
        self.custom_data_root = custom_dataRoot
        self.sentence_encoder = sentence_encoder
        self.custom_data_dir = osp.join(self.custom_data_root, f"pubmed_{self.sentence_encoder.name}")

        if not osp.exists(self.custom_data_dir):
            os.makedirs(self.custom_data_dir)

        super().__init__(self.custom_data_dir, transform, pre_transform, pre_filter)

        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ["data.pt", "texts.pkl"]

    def text_to_embed(self, texts):
        if self.sentence_encoder is None:
            raise NotImplementedError("Sentence Encoder is not passed")
        if texts is None:
            return None
        else:
            # returns to self.device
            return self.sentence_encoder.encode(texts)

    def encode_texts(self, texts):
        if isinstance(texts[0], str):
            return self.text_to_embed(texts)
        return [self.text_to_embed(t) for t in texts]

    def generate_custom_data(self):
        # Load the raw pubmed dataset
        data_path = osp.join(self.data_root, "pubmed", "pubmed.pt")
        raw_pubmed_data = torch.load(data_path)

        texts = raw_pubmed_data.raw_text
        label_names = raw_pubmed_data.label_names


        # Label and label description
        category_desc = pd.read_csv(osp.join(self.data_root, "pubmed", "categories.csv"), sep=",", header=None).values

        # Sort the label desc by the order of label_names
        ordered_desc = []
        for i, label in enumerate(label_names):
            true_ind = (label == category_desc[:, 0])
            ordered_desc.append((label, category_desc[true_ind, 1][0]))

        # Prompts for nodes/edges in original graph (can be changed accordingly)
        node_texts = ["Feature Node.\n Paper Title and abstract: " + t for t in texts]
        edge_text = ["Feature Edge.\n Connected papers are cited together by other papers."]

        # Node classification : Prompts for prompt node and label node (can be changed accordingly)
        prompt_node_text = ["Prompt Node.\n Node Classification on the paper's category"]
        label_texts = ["Prompt Node.\n Literature Category and Description: " + desc[0] + " + " + desc[1] for desc in ordered_desc]

        # Link prediction : Prompts for prompt node and edge labels (can be changed accordingly)
        prompt_node_edge_text = ["Prompt Node.\n Link Prediction on the papers that are cited together"]
        edge_label_text = ["Prompt Node.\n Two papers have co-citation",
                           "Prompt Node.\n Two papers do not have co-citation"]

        # Prompt for edge b/w prompt node and labels (can be changed accordingly)
        prompt_edge_text = ["Prompt Edge."]

        return raw_pubmed_data, [node_texts, label_texts, edge_text, prompt_node_edge_text, prompt_node_text, prompt_edge_text, edge_label_text]

    def process(self):
        # raw pubmed dataset is not in any library, so we process and load it manually in self.generate_custom_data()
        pubmed_data_list, texts = self.generate_custom_data()
        texts_embed = self.encode_texts(texts)

        torch.save(texts, self.processed_paths[1])

        pubmed_data_list.num_nodes = pubmed_data_list.y.shape[0]


        pubmed_data_list.x_text_feat = texts_embed[0]
        pubmed_data_list.label_text_feat = texts_embed[1]
        pubmed_data_list.edge_text_feat = texts_embed[2]
        pubmed_data_list.prompt_text_edge_feat = texts_embed[3]
        pubmed_data_list.prompt_text_feat = texts_embed[4]
        pubmed_data_list.prompt_edge_feat = texts_embed[5]
        pubmed_data_list.edge_label_feat = texts_embed[6]

        pubmed_data_list.train_mask = pubmed_data_list.train_masks[0]
        pubmed_data_list.val_mask = pubmed_data_list.val_masks[0]
        pubmed_data_list.test_mask = pubmed_data_list.test_masks[0]

        pubmed_data_list.train_masks = None
        pubmed_data_list.val_masks = None
        pubmed_data_list.test_masks = None

        # Pass the data_list as a list
        data, slices = self.collate([pubmed_data_list])

        torch.save((data, slices), self.processed_paths[0])
        print("Pubmed is processed. Saved.")

In [16]:
LMencoder = SentenceEncoder("ST", root="../lang_models", device=2)
custom_pubmed = PubmedPyGDataset(dataRoot=data_root, sentence_encoder=LMencoder)

In [17]:
LMencoder2 = SentenceEncoder("roberta", root="../lang_models", device=2)
custom_pubmed2 = PubmedPyGDataset(dataRoot=data_root, sentence_encoder=LMencoder2)

In [18]:
custom_pubmed

PubmedPyGDataset()

In [19]:
pubmed = custom_pubmed._data
pubmed

Data(x=[19717, 384], edge_index=[2, 88670], y=[19717], label_names=[3], category_names=[19717], raw_text=[19717], num_nodes=19717, x_text_feat=[19717, 768], label_text_feat=[3, 768], edge_text_feat=[1, 768], prompt_text_edge_feat=[1, 768], prompt_text_feat=[1, 768], prompt_edge_feat=[1, 768], edge_label_feat=[2, 768], train_mask=[19717], val_mask=[19717], test_mask=[19717])

In [20]:
for k, v in pubmed:
    print(k, type(v))

x <class 'torch.Tensor'>
edge_index <class 'torch.Tensor'>
y <class 'torch.Tensor'>
label_names <class 'list'>
category_names <class 'list'>
raw_text <class 'list'>
num_nodes <class 'int'>
x_text_feat <class 'torch.Tensor'>
label_text_feat <class 'torch.Tensor'>
edge_text_feat <class 'torch.Tensor'>
prompt_text_edge_feat <class 'torch.Tensor'>
prompt_text_feat <class 'torch.Tensor'>
prompt_edge_feat <class 'torch.Tensor'>
edge_label_feat <class 'torch.Tensor'>
train_mask <class 'torch.Tensor'>
val_mask <class 'torch.Tensor'>
test_mask <class 'torch.Tensor'>


In [21]:
pubmed.is_directed()

False

: 